In [36]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import datetime as dt

%matplotlib inline

# dit is nodig om jupyter in widescreen te gebruiken (#UWHD masterrace:)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Use 0 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

In [37]:
# meetpost_id's per richting
#----------------------------
meetpunten_riGent = [197, 198, 199, 3744, 3745, 3746, 3750, 3751, 3752, 3756, 3757, 3758, 499, 500, 501]
meetpunten_riGent_lane1 = [197, 3744, 3750, 3756, 499]
meetpunten_riGent_lane2 = [198, 3745, 3751, 3757, 500]
meetpunten_riGent_lane3 = [199, 3746, 3752, 3758, 501]
meetpunten_riAnt = [200, 201, 202, 3747, 3748, 3749, 3753, 3754, 3755, 3759, 3760, 3761, 496, 497, 498, 3691, 3692, 3693, 3694]
meetpunten_riAnt_lane1 = [200, 3747, 3753, 3759, 496, 3691]
meetpunten_riAnt_lane2 = [201, 3748, 3754, 3760, 497, 3692]
meetpunten_riAnt_lane3 = [202, 3749, 3755, 3761, 498, 3693]

loop_richting_en_rijstroken = {"Richting Gent - rijstrook 1": meetpunten_riGent_lane1,
                               "Richting Gent - rijstrook 2": meetpunten_riGent_lane2,
                               "Richting Gent - rijstrook 3": meetpunten_riGent_lane3,
                               "Richting Antwerpen - rijstrook 1": meetpunten_riAnt_lane1,
                               "Richting Antwerpen - rijstrook 2": meetpunten_riAnt_lane2,
                               "Richting Antwerpen - rijstrook 3": meetpunten_riAnt_lane3,
                               }

loop_richting_en_rijstroken_ANT = {"Richting Antwerpen - rijstrook 1": meetpunten_riAnt_lane1,
                                   "Richting Antwerpen - rijstrook 2": meetpunten_riAnt_lane2,
                                   "Richting Antwerpen - rijstrook 3": meetpunten_riAnt_lane3,
                                   }

# loop_richting_en_rijstroken_ANT = {"Richting Antwerpen - rijstrook 1": meetpunten_riAnt_lane1}
meetpunten_riGENT_1lus_allerijstroken = {"Richting Gent - Eerste meetlus - 3 rijstroken": [3744,3745,3746]}
meetpunten_riANT_1lus_allerijstroken = {"Richting Antwerpen - Eerste meetlus - 3 rijstroken": [3747,3748,3749]}
meetpunten_riAnt_alle = {"Richting Antwerpen - alle lussen - alle rijstroken": meetpunten_riAnt}

In [38]:
# Koppeling kmpunten naar meetpuntid's:
# --------------------------------------
gent_r1_dict = {13.517 : '197', 13.72 : '3744', 13.921 : '3750', 14.109 : '3756', 14.274 : '499'}
gent_r2_dict = {13.517 : '198', 13.72 : '3745', 13.921 : '3751', 14.109 : '3757', 14.274 : '500'}
gent_r3_dict = {13.517 : '199', 13.72 : '3746', 13.921 : '3752', 14.109 : '3758', 14.274 : '501'}

ant_r1_dict = {13.518 : '200', 13.719 : '3747', 13.918 : '3753', 14.106 : '3759', 14.27 : '496', 14.669 : '3691'}
ant_r2_dict = {13.518 : '201', 13.719 : '3748', 13.918 : '3754', 14.106 : '3760', 14.27 : '497', 14.669 : '3692'}
ant_r3_dict = {13.518 : '202', 13.719 : '3749', 13.918 : '3755', 14.106 : '3761', 14.27 : '498', 14.669 : '3693'}

In [39]:
# Lees de koppeltabel in (csv1) en de ongevallendata (csv2)
# ----------------------------------------------------------
csv1 = 'C:\Case\TargetData\Koppeltabel_targetdata_csv1.csv'
csv2 = 'C:\Case\TargetData\Koppeltabel_targetdata_csv2_v2.csv'

df1 = pd.read_csv(csv1,sep=';', dayfirst=True)
df1.set_index('meetpunt_id',inplace=True)

df2 = pd.read_csv(csv2,sep=';', dayfirst=True)
df2.set_index('Array nr',inplace=True)

In [40]:
df1

,positie_rsys_hrb,ident_8_rsys_hrb,lane,Richting,kmpunt_afgerond,meetpost_naam
meetpunt_id,,,,,,
197,13.517,R0010001,1,GENT,13.5,13.5_Lane1_GENT
3744,13.720,R0010001,1,GENT,13.7,13.7_Lane1_GENT
3750,13.921,R0010001,1,GENT,13.9,13.9_Lane1_GENT
3756,14.109,R0010001,1,GENT,14.1,14.1_Lane1_GENT
499,14.274,R0010001,1,GENT,14.3,14.3_Lane1_GENT
...,...,...,...,...,...,...
3755,13.918,R0010002,3,ANT,13.9,13.9_Lane3_ANT
3761,14.106,R0010002,3,ANT,14.1,14.1_Lane3_ANT
498,14.270,R0010002,3,ANT,14.3,14.3_Lane3_ANT


In [41]:
df2

,Datum,Start Tijdstip,Einde Tijdstip,Ident8,Rijstrook,Kmpunt obv camera
Array nr,,,,,,
1,2015-05-06,5:43,6:24,A0010002,1,13.580
7,2015-10-12,22:18,23:06,A0010001,"1,2,3",13.600
9,2015-11-23,9:41,10:27,R0010001,"1,2",14.100
10,2016-02-01,14:15,15:23,R0010002,1,14.205
1,2016-02-05,11:34,13:38,R0010002,1,13.720
...,...,...,...,...,...,...
43,2017-08-11,13:14,13:26,R0010001,1,13.450
44,2017-09-04,10:55,12:27,R0010002,"1,2",13.950
46,2017-10-02,22:05,22:07,R0010002,3,13.800


In [42]:
def maakTijdRange(startString, eindString, freq='5min'):
    # maakt een 'tijdrange' tussen twee tijddstippen, volgens bepaalde freq (standaard 5min)
    
    startTijd = pd.to_datetime(startString).floor(freq)
    eindTijd = pd.to_datetime(eindString).floor(freq)   # hier toch geen ceil gebruiken, want de laatste waarde in de result array moet 
                                                        #vòòr de niet-afgeronde tijd komen (anders tel je 5 min tveel op het einde)
    
    return pd.period_range(start = startTijd, end = eindTijd, freq = freq)

In [43]:
def zoekNearestMeetpunt(mydict,kmpunt):
    # zoekt dichtstbijzijnde kmpunt in de dict values (=kmpunt van meetpuntid) en returnt de key (=meetpuntid)
    
    return (mydict[min(mydict, key=lambda x:abs(x-kmpunt))])

In [44]:
def maakKoppeling(ident8, rijstrook, kmpunt):
    # zoekt meetpunt_id dat bij de richting (ident8), rijstrook en dichtstbijzijnde kmpunt behoort
    
    if ident8 == 'R0010001' or ident8 == 'A0010001':
        # richting GENT
        if rijstrook == '1':
            meetpunt_id = zoekNearestMeetpunt(gent_r1_dict,kmpunt)
        elif rijstrook == '2':
            meetpunt_id = zoekNearestMeetpunt(gent_r2_dict,kmpunt)
        elif rijstrook == '3':
            meetpunt_id = zoekNearestMeetpunt(gent_r3_dict,kmpunt)
        else:
            print('rijstrook NIET GELDIG?!: ' + rijstrook)
        
    elif ident8 == 'R0010002' or ident8 == 'A0010002': 
        # richting ANTWERPEN
        if rijstrook == '1':
            meetpunt_id = zoekNearestMeetpunt(ant_r1_dict,kmpunt)            
        elif rijstrook == '2':
            meetpunt_id = zoekNearestMeetpunt(ant_r2_dict,kmpunt)
        elif rijstrook == '3':
            meetpunt_id = zoekNearestMeetpunt(ant_r3_dict,kmpunt)
        else:
            print('rijstrook NIET GELDIG?!: ' + rijstrook)
        
    else:
        print('IDENT8 nummer NIET GELDIG?!: ' + ident8)
     
    return meetpunt_id

In [45]:
rows_list = []

for row in df2.itertuples(): 
    arrayNr, datum, startTijd, eindTijd, ident8, rijstroken, kmpunt = row
    
    startDate = pd.to_datetime(datum+ ' ' + startTijd)
    endDate = pd.to_datetime(datum+ ' ' + eindTijd)
    
    print('bezig met array nr: ' + str(arrayNr))
    
    for tijd in maakTijdRange(startDate,endDate):
        for rijstrook in rijstroken.split(","):
            if rijstrook != '0':
                meetpunt_id = maakKoppeling(ident8, rijstrook, kmpunt)
                print('\t' + str(tijd) +' - '+ meetpunt_id)
                rows_list.append(tijd.strftime('%Y-%m-%d %H:%M:%S') + ' - ' + meetpunt_id)

                
target_df = pd.DataFrame(rows_list,columns=['targetData'])
target_df.to_csv(r'C:\Case\TargetData\targetdata_output_02.csv', index=False)

bezig met array nr: 1
	2015-05-06 05:40 - 200
	2015-05-06 05:45 - 200
	2015-05-06 05:50 - 200
	2015-05-06 05:55 - 200
	2015-05-06 06:00 - 200
	2015-05-06 06:05 - 200
	2015-05-06 06:10 - 200
	2015-05-06 06:15 - 200
	2015-05-06 06:20 - 200
bezig met array nr: 7
	2015-10-12 22:15 - 197
	2015-10-12 22:15 - 198
	2015-10-12 22:15 - 199
	2015-10-12 22:20 - 197
	2015-10-12 22:20 - 198
	2015-10-12 22:20 - 199
	2015-10-12 22:25 - 197
	2015-10-12 22:25 - 198
	2015-10-12 22:25 - 199
	2015-10-12 22:30 - 197
	2015-10-12 22:30 - 198
	2015-10-12 22:30 - 199
	2015-10-12 22:35 - 197
	2015-10-12 22:35 - 198
	2015-10-12 22:35 - 199
	2015-10-12 22:40 - 197
	2015-10-12 22:40 - 198
	2015-10-12 22:40 - 199
	2015-10-12 22:45 - 197
	2015-10-12 22:45 - 198
	2015-10-12 22:45 - 199
	2015-10-12 22:50 - 197
	2015-10-12 22:50 - 198
	2015-10-12 22:50 - 199
	2015-10-12 22:55 - 197
	2015-10-12 22:55 - 198
	2015-10-12 22:55 - 199
	2015-10-12 23:00 - 197
	2015-10-12 23:00 - 198
	2015-10-12 23:00 - 199
	2015-10-12 23:05 - 

In [46]:
target_df

,targetData
0,2015-05-06 05:40:00 - 200
1,2015-05-06 05:45:00 - 200
2,2015-05-06 05:50:00 - 200
3,2015-05-06 05:55:00 - 200
4,2015-05-06 06:00:00 - 200
...,...
514,2017-10-26 10:45:00 - 3745
515,2017-10-26 10:45:00 - 3746
516,2017-10-26 10:50:00 - 3744
517,2017-10-26 10:50:00 - 3745


In [49]:
df_set2 = pd.read_csv('C:\Case\TargetData\Dataset1_selectie_ongevallen_timeframe2u.csv', dayfirst=True)


df_set2["datetime"] = pd.to_datetime(df_set2["datetime"], dayfirst=True)
df_set2.set_index('datetime',inplace=True)


df_model = df_set2.filter(["datetime"])

df_model['date'] = df_model.index.strftime('%Y-%m-%d')
df_model['time'] = df_model.index.strftime('%H:%M:%S')

df_model

tijdArray = []

for row in df_model.itertuples():
    datetime, date, time = row
    
    startTime = (datetime - dt.timedelta(hours=1))
    endTime = (datetime + dt.timedelta(hours=1))

    periode = maakTijdRange(startTime, endTime, freq='5min')
    
    for time in periode:
        tijdArray.append(time)
    
df_result =pd.DataFrame({'Selectie':tijdArray})

df_result

df_result.to_csv(r'C:\Case\TargetData\selectie_perioden_01.csv', index=False)


In [50]:
df_model

,date,time
datetime,,
2015-08-28 15:11:00,2015-08-28,15:11:00
2015-08-28 15:11:00,2015-08-28,15:11:00
2015-09-15 13:53:00,2015-09-15,13:53:00
2015-11-23 09:41:00,2015-11-23,09:41:00
2016-02-24 10:28:00,2016-02-24,10:28:00
2016-05-25 14:21:00,2016-05-25,14:21:00
2016-06-30 15:23:00,2016-06-30,15:23:00
2016-07-04 09:55:00,2016-07-04,09:55:00
2016-09-27 11:25:00,2016-09-27,11:25:00
